# Plotting of the Definitions of Timing Uncertainties

In [ ]:
import os,sys
import numpy as np
import scipy.stats 

from scipy import signal

In [ ]:
%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt
plt.rcParams["figure.dpi"] = 300

In [ ]:
# some slightly more fancy axes 
# https://stackoverflow.com/questions/33737736/matplotlib-axis-arrow-tip
def arrowed_spines(fig, ax):

    xmin, xmax = ax.get_xlim() 
    ymin, ymax = ax.get_ylim()

    # removing the default axis on all sides:
    for side in ['bottom','right','top','left']:
        ax.spines[side].set_visible(False)

    # removing the axis ticks
    plt.xticks([]) # labels 
    plt.yticks([])
    ax.xaxis.set_ticks_position('none') # tick markers
    ax.yaxis.set_ticks_position('none')

    # get width and height of axes object to compute 
    # matching arrowhead length and width
    dps = fig.dpi_scale_trans.inverted()
    bbox = ax.get_window_extent().transformed(dps)
    width, height = bbox.width, bbox.height

    # manual arrowhead width and length
    hw = 1./20.*(ymax-ymin) 
    hl = 1./20.*(xmax-xmin)
    lw = 1. # axis line width
    ohg = 0.3 # arrow overhang

    # compute matching arrowhead length and width
    yhw = hw/(ymax-ymin)*(xmax-xmin)* height/width 
    yhl = hl/(xmax-xmin)*(ymax-ymin)* width/height

    # draw x and y axis
    ax.arrow(xmin, 0, xmax-xmin, 0., fc='k', ec='k', lw = lw, 
             head_width=hw, head_length=hl, overhang = ohg, 
             length_includes_head= True, clip_on = False) 

    ax.arrow(0, ymin, 0., ymax-ymin, fc='k', ec='k', lw = 3*lw, 
             head_width=yhw, head_length=yhl, overhang = ohg, 
             length_includes_head= True, clip_on = False)


### Clock Skew

In [ ]:
ymin = -7
ymax = 2

# draw reference value lines
pt00 = 0.2
plt.plot([pt00,pt00],[ymin+2,ymax], '--', linewidth=1,color='black')
pt01 = 0.25
plt.plot([pt01,pt01],[ymin+2,ymax], '--', linewidth=1,color='black')

# plot arrows to denote the skew
plt.arrow(0.04,ymin+2.5,0.15,0, head_width = 0.2,head_length=0.05,length_includes_head=True,color='black')
plt.arrow(0.41,ymin+2.5,-0.15,0, head_width = 0.2,head_length=0.05,length_includes_head=True,color='black')
plt.text(0.1,ymin+1,'skew', color='black',fontsize=16)

# plot clocks
t = np.linspace(0.01, 3.01, 1000, endpoint=False)
plt.plot(t, signal.square(-np.pi * 5 * t), color='black')
plt.text(-0.3,-1,'CLK1', color='black',fontsize=16)
plt.plot(t+0.05, signal.square(-np.pi * 5 * t)-2.5, color='black')
plt.text(-0.3,-3.5,'CLK2', color='black',fontsize=16)

plt.ylim(-6, ymax)
plt.xlim(-0.5,1.7)
plt.axis('off')

plt.savefig('../../skew.eps', transparent=False)

### Error definitions according to wikipedia

* __Accuracy__ is the proximity of measurement results to the true (reference) value.
* __Precision__ is the degree to which repeated (or reproducible) measurements under unchanged conditions show the same results.

### Plot the skew measurements of a slave clock

In [ ]:
x_min = -5
x_max = 80.0

plt.figure(figsize=(10,5),frameon=False)

mean = 50.0 
std = 6.0

x = np.linspace(x_min, x_max, 1000)
y = scipy.stats.norm.pdf(x,mean,std)

pt0 = mean
ymax = scipy.stats.norm.pdf(pt0 ,mean, std)

plt.plot(x,y, color='black')
fig = plt.gcf()

plt.ylim(top=1.2*ymax)
ax = plt.gca()
arrowed_spines(fig, ax)

# label axes
plt.text(x_max-10,-0.01, 'Time Error', fontsize=16)
plt.text(x_min-3, 1.15*ymax, 'Number of Measurements', fontsize=16, rotation='vertical')

# draw mean
plt.plot([pt0,pt0],[0.0,ymax], color='grey')
plt.text(pt0-2,1.05*ymax,'Tested\nClock')

# draw reference value
pt00 = 10
plt.plot([pt00,pt00],[0.0,ymax], color='black')
plt.text(pt00-3,1.05*ymax,'Reference\nClock')

# draw rms on mean
pt1 = mean + std
pt2 = mean - std

ptx = np.linspace(pt1, pt2, 10)
pty = scipy.stats.norm.pdf(ptx,mean,std)

#plt.fill_between(ptx, pty, alpha='0.3')
plt.fill_between(ptx, pty, color='lightblue')

# draw min/max values
ptmax = mean + 4*std
ptmin = mean - 4*std

plt.plot([ptmax,ptmax],[0.0,0.2*ymax], color='black')
plt.text(ptmax-3,0.25*ymax,r'max($\tau$)')
plt.plot([ptmin,ptmin],[0.0,0.2*ymax], color='black')
plt.text(ptmin-3,0.25*ymax,r'min($\tau$)')

# annotate definitions
plt.plot([pt00,pt0],[0.85*ymax,0.85*ymax], color='mediumblue') 
plt.text(pt00+13,0.9*ymax,'Mean Accuracy', color='mediumblue')

ptsigma = pty[0]
plt.plot([pt1,pt2],[ptsigma,ptsigma], color='mediumblue')
plt.text(pt0+7,0.62*ymax,'Precision', color='mediumblue')

plt.plot([ptmin,ptmax],[0.1*ymax,0.1*ymax], color='mediumblue') 
plt.text(pt0-23,0.13*ymax,r'MTIE($\tau$)', color='mediumblue')

ptsigma = pty[0]
plt.text(pt0+2,1.05*ptsigma,r'$\sigma$')
plt.text(pt0-3,1.05*ptsigma,r'$\sigma$')
plt.text(pt0-3,-0.1*ymax,'mean')

#plt.savefig('../../AccuracyAndPrecision.eps', transparent=False)

### Definitions borrowed from the WR Torture Report

* The mean of the measured PPS skew value between the Reference Clock and the Clock Under Test is interpreted as __accuracy__. 
* The standard deviation of the mesurement ($\sigma$) is interpreted as __precision__, meaning that 64.2% of the data samples fall into the range of $mean \pm \sigma$. 
* The (PPS) skew, in the literature, is also called __Time Error (TE)__.
* The __Time Interval Error (TIE)__ is defined as the phase difference between the signal being measured and the reference clock and is conventionally set to zero at the start of the total measurement period $	au$. Therefore TIE gives the phase change since the measurement began. 
* The __Maximum Time Interval Error (MTIE)__ is the maximum peak-to-peak variation (or jitter) of the clock synchronization during the "observation" time window $\tau$. MTIE is a measure of wander that characterizes frequency offsets and phase transients.

### Plot the skew measurement of several slave clocks

In [ ]:
# definition of root mean square error
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [ ]:
x_min = -5
x_max = 150.0

plt.figure(figsize=(14,6),frameon=False)

mean = 80.0 
std = 6.0

x = np.linspace(x_min, x_max, 1000)
y = scipy.stats.norm.pdf(x,mean,std)

pt0 = mean
ymax = scipy.stats.norm.pdf(pt0 ,mean, std)

plt.plot(x,y, color='black')

fig = plt.gcf()
plt.ylim(top=1.2*ymax)
ax = plt.gca()
arrowed_spines(fig, ax)

# label axes
plt.text(x_max-10,-0.01, 'Time Error', fontsize=16)
plt.text(x_min-2,1.15*ymax, 'Number of samples', fontsize=16, rotation='vertical')

# draw mean
plt.plot([pt0,pt0],[0.0,ymax], color='grey')
plt.text(pt0-4,1.05*ymax,'Tested \nClock 1')

# draw reference value
pt00 = 50
plt.plot([pt00,pt00],[0.0,ymax], color='red')
plt.text(pt00-3,1.05*ymax,'Reference \nClock', color='red')

# draw rms on mean
pt1 = mean + std
pt2 = mean - std

ptx = np.linspace(pt1, pt2, 10)
pty = scipy.stats.norm.pdf(ptx,mean,std)

#plt.fill_between(ptx, pty, alpha='0.3', color='grey')
plt.fill_between(ptx, pty, color='lightgrey')

# ------------------------------------------------------------------------------------------------------
# add a second slave to the plot
mean2 = 25.0 
std2 = 9.0

x2 = np.linspace(x_min, x_max, 1000)
y2 = scipy.stats.norm.pdf(x2,mean2,std2)
ymax2 = scipy.stats.norm.pdf(mean2 ,mean2, std2)

plt.plot(x2,y2, color='black')

# draw mean
plt.plot([mean2,mean2],[0.0,ymax2], color='grey')
plt.text(mean2-4,0.72*ymax,'Tested \nClock 2')

# draw rms on mean
pt12 = mean2 + std2
pt22 = mean2 - std2

ptx2 = np.linspace(pt12, pt22, 10)
pty2 = scipy.stats.norm.pdf(ptx2,mean2,std2)

#plt.fill_between(ptx2, pty2, alpha='0.3', color='grey')
plt.fill_between(ptx2, pty2, color='lightgrey')

# annotate definitions
#plt.plot([pt00,mean2],[0.48*ymax,0.48*ymax], color='mediumblue') 
#plt.text(mean2+8,0.52*ymax,'Accuracy\n(Clock 2)', color='mediumblue')

# ------------------------------------------------------------------------------------------------------
# add a third slave to the plot
mean3 = 110.0 
std3 = 7.5

x3 = np.linspace(x_min, x_max, 1000)
y3 = scipy.stats.norm.pdf(x3,mean3,std3)
ymax3 = scipy.stats.norm.pdf(mean3 ,mean3, std3)

plt.plot(x3,y3, color='black')

# draw mean
plt.plot([mean3,mean3],[0.0,ymax3], color='grey')
plt.text(mean3-4,0.85*ymax,'Tested \nClock 3')

# draw rms on mean
pt13 = mean3 + std3
pt23 = mean3 - std3

ptx3 = np.linspace(pt13, pt23, 10)
pty3 = scipy.stats.norm.pdf(ptx3,mean3,std3)

#plt.fill_between(ptx3, pty3, alpha='0.3', color='grey')
plt.fill_between(ptx3, pty3, color='lightgrey')

# ------------------------------------------------------------------------------------------------------
# get the rms accuracy defined

# values of slaves clocks
acc = np.array([mean,mean2,mean3])
acc_error = np.array([std,std2,std3])

allmean = np.mean(acc)

plt.plot([allmean,allmean],[0.0,ymax], color='mediumblue',lw=2)
plt.text(allmean-6,1.05*ymax,'Tested \nClocks', color='mediumblue')

#allstd = np.std(acc)
#allaverage = np.average(acc,weights=acc_error)
#plt.plot([allaverage,allaverage],[0.0,ymax], color='mediumblue',lw=2)
#plt.text(allaverage-6,1.05*ymax,'Average \n(slaves)', color='mediumblue')

# RMSE values of master/reference clock
master = np.full(len(acc),pt00)
allrmse = rmse(acc,master)
plt.plot([pt00-allrmse,pt00+allrmse],[-0.01*ymax,-0.01*ymax], color='red',lw=4)
plt.text(allrmse,-0.13*ymax,'Reference \nClock $\pm$ RMSE', color='red')

# RMSE values of mean slave clock
master = np.full(len(acc),allmean)
allrmse = rmse(acc,master) # this is the same value as the np.std(acc)
plt.plot([allmean-allrmse,allmean+allrmse],[0.01*ymax,0.01*ymax], color='mediumblue',lw=4)
plt.text(allrmse+30,-0.13*ymax,'Tested Clock \nMean$\pm$ STD', color='mediumblue')

#plt.savefig('../../RMSAccuracy.eps', transparent=False)

### Further definitions

In order to estimate the overall accuracy of the clock distribution system, i.e. a setup with more then one slave clock as system under test, the __root-mean-square error (RMSE)__ is used. It is defined as the square root of the quadratic mean of differences between predicted (true) values $T$ and observed values $M$, i.e., 

RMSE $= \sqrt {\frac{1}{N} \sum _{i=1}^{N}(M_i-T)^{2}}$.

In general, a lower RMSE value is better than a higher one. It should be noted that the effect of each error on RMSE is proportional to the size of the squared error; thus larger errors have a disproportionately large effect on RMSE and consequently, RMSE is sensitive to outliers. Furthermore, the RMSE value equals the standard deviation of the mean of all slaves, if the "true" value is the mean itself. 